# 대리점 데이터를 가공하는 테크닉

#### 데이터 읽기

In [1]:
import pandas as pd

In [2]:
uriage = pd.read_csv("./pyda100-master/2장/uriage.csv")
uriage

,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02,상품A,100.0,김가온
1,2019-07-13 13:05,상 품 S,NaN,김우찬
2,2019-05-11 19:42,상 품 a,NaN,김유찬
3,2019-02-12 23:40,상품Z,2600.0,김재현
4,2019-04-22 3:09,상품a,NaN,김강현
...,...,...,...,...
2994,2019-02-15 2:56,상품Y,2500.0,김정민
2995,2019-06-22 4:03,상품M,1300.0,김재원
2996,2019-03-29 11:14,상품Q,NaN,김지율
2997,2019-07-14 12:56,상품H,NaN,김승주


In [4]:
kokyaku_daicho = pd.read_excel("./pyda100-master/2장/kokyaku_daicho.xlsx")
kokyaku_daicho

,고객이름,지역,등록일
0,김 현성,H시,2018-01-04 00:00:00
1,김 도윤,E시,42782
2,김 지한,A시,2018-01-07 00:00:00
3,김 하윤,F시,42872
4,김 시온,E시,43127
...,...,...,...
195,김 재희,G시,2017-06-20 00:00:00
196,김 도영,E시,2018-06-20 00:00:00
197,김 이안,F시,2017-04-29 00:00:00
198,김 시현,H시,2019-04-19 00:00:00


#### 데이터 오류를 살펴보자.

In [7]:
uriage['item_name'].head(10)

0      상품A
1    상 품 S
2    상 품 a
3      상품Z
4      상품a
5      상품S
6     상품 a
7      상품V
8      상품O
9      상품A
Name: item_name, dtype: object

In [11]:
uriage['purchase_date'] = pd.to_datetime(uriage['purchase_date'])
uriage['purchase_month'] = uriage['purchase_date'].dt.strftime("%Y-%m")
uriage.pivot_table(index='purchase_month', columns='item_name', values='item_price', aggfunc='sum', fill_value=0)

item_name,상 품 n,상품 E,상품 M,상품 P,상품 S,상품 W,상품 X,상품W,상 품O,상 품Q,...,상품k,상품l,상품o,상품p,상품r,상품s,상품t,상품v,상품x,상품y
purchase_month,,,,,,,,,,,,,,,,,,,,,
2019-01,1400,0,0,0,0,0,0,0,0,0,...,1100,1200,1500,0,0,0,0,0,0,0
2019-02,0,0,0,0,0,0,2400,0,0,0,...,0,0,0,0,0,1900,2000,2200,0,0
2019-03,0,500,1300,1600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2019-04,0,0,0,0,0,0,0,2300,0,1700,...,0,0,0,0,0,1900,0,0,0,0
2019-05,0,0,0,0,1900,0,0,0,0,0,...,0,1200,0,0,0,0,0,0,0,2500
2019-06,0,0,0,0,0,2300,0,0,0,0,...,0,0,0,1600,0,0,0,0,2400,0
2019-07,0,0,0,0,0,0,0,0,0,0,...,0,0,1500,0,1800,0,0,0,0,0


#### 오류 수정(정합성)

In [13]:
len(uriage['item_name'].unique()) #26

99

In [14]:
uriage['item_name'] = uriage['item_name'].str.upper()
uriage['item_name'] = uriage['item_name'].str.replace(" ", "")
uriage['item_name'].head()

0    상품A
1    상품S
2    상품A
3    상품Z
4    상품A
Name: item_name, dtype: object

In [16]:
len(uriage['item_name'].unique())

26

In [17]:
uriage['item_name'].unique()

array(['상품A', '상품S', '상품Z', '상품V', '상품O', '상품U', '상품L', '상품C', '상품I',
       '상품R', '상품X', '상품G', '상품P', '상품Q', '상품Y', '상품N', '상품W', '상품E',
       '상품K', '상품B', '상품F', '상품D', '상품M', '상품H', '상품T', '상품J'],
      dtype=object)

#### 금액의 결측치 수정

In [18]:
uriage.isnull().sum()

purchase_date       0
item_name           0
item_price        387
customer_name       0
purchase_month      0
dtype: int64

In [21]:
len(uriage.item_price.unique())

27

In [30]:
uriage.item_price.unique()

array([ 100.,   nan, 2600., 1900., 2200., 1500., 2100., 1200.,  300.,
        900., 1800., 2400.,  700., 1600., 1700., 2500., 2300.,  500.,
       1100.,  200.,  600., 1400.,  400., 1300.,  800., 2000., 1000.])

In [33]:
flg_is_null = uriage['item_price'].isnull()
for trg in list(uriage.loc[flag_is_null, 'item_name'].unique()) : 
    price = uriage.loc[(~flag_is_null) & (uriage['item_name'] == trg), 'item_price'].max()
    uriage['item_price'].loc[(flg_is_null) & (uriage['item_name'] == trg)] = price
uriage.head()

,purchase_date,item_name,item_price,customer_name,purchase_month
0,2019-06-13 18:02:00,상품A,100.0,김가온,2019-06
1,2019-07-13 13:05:00,상품S,1900.0,김우찬,2019-07
2,2019-05-11 19:42:00,상품A,100.0,김유찬,2019-05
3,2019-02-12 23:40:00,상품Z,2600.0,김재현,2019-02
4,2019-04-22 03:09:00,상품A,100.0,김강현,2019-04


In [34]:
len(uriage.item_price.unique())

26

In [35]:
uriage.item_price.unique()

array([ 100., 1900., 2600., 2200., 1500., 2100., 1200.,  300.,  900.,
       1800., 2400.,  700., 1600., 1700., 2500., 1400., 2300.,  500.,
       1100.,  200.,  600.,  400., 1300.,  800., 2000., 1000.])

In [37]:
uriage.isnull().sum()[uriage.isnull().sum()>0]

Series([], dtype: int64)

In [51]:
tmp = pd.DataFrame(uriage.groupby('item_name')['item_price'].min()) #skipna=False 옵션을 사용을 못해서.. 이 방법은 X
tmp = pd.merge(tmp, pd.DataFrame(uriage.groupby('item_name')['item_price'].max()), on='item_name')
tmp

,item_price_x,item_price_y
item_name,,
상품A,100.0,100.0
상품B,200.0,200.0
상품C,300.0,300.0
상품D,400.0,400.0
상품E,500.0,500.0
상품F,600.0,600.0
상품G,700.0,700.0
상품H,800.0,800.0
상품I,900.0,900.0


In [50]:
for trg in list(uriage['item_name'].sort_values().unique()) :
    print(trg + "\t최고가 : " + str(uriage.loc[uriage['item_name']==trg]['item_price'].max()) + "\t최저가 : " + str(uriage.loc[uriage['item_name']==trg]['item_price'].min(skipna=False)))

상품A	최고가 : 100.0	최저가 : 100.0
상품B	최고가 : 200.0	최저가 : 200.0
상품C	최고가 : 300.0	최저가 : 300.0
상품D	최고가 : 400.0	최저가 : 400.0
상품E	최고가 : 500.0	최저가 : 500.0
상품F	최고가 : 600.0	최저가 : 600.0
상품G	최고가 : 700.0	최저가 : 700.0
상품H	최고가 : 800.0	최저가 : 800.0
상품I	최고가 : 900.0	최저가 : 900.0
상품J	최고가 : 1000.0	최저가 : 1000.0
상품K	최고가 : 1100.0	최저가 : 1100.0
상품L	최고가 : 1200.0	최저가 : 1200.0
상품M	최고가 : 1300.0	최저가 : 1300.0
상품N	최고가 : 1400.0	최저가 : 1400.0
상품O	최고가 : 1500.0	최저가 : 1500.0
상품P	최고가 : 1600.0	최저가 : 1600.0
상품Q	최고가 : 1700.0	최저가 : 1700.0
상품R	최고가 : 1800.0	최저가 : 1800.0
상품S	최고가 : 1900.0	최저가 : 1900.0
상품T	최고가 : 2000.0	최저가 : 2000.0
상품U	최고가 : 2100.0	최저가 : 2100.0
상품V	최고가 : 2200.0	최저가 : 2200.0
상품W	최고가 : 2300.0	최저가 : 2300.0
상품X	최고가 : 2400.0	최저가 : 2400.0
상품Y	최고가 : 2500.0	최저가 : 2500.0
상품Z	최고가 : 2600.0	최저가 : 2600.0
